## Grab links for jits

### Outputs (TODO)

- PAPERS_INFO/JITS/JITS_ISSUE_LINKS
- PAPERS_INFO/JITS/JITS_PAPER_LINKS

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import time
import sqlite3 as sql

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [22]:
# journal of intelligent transportation systems
base_url = 'https://www.tandfonline.com/toc/gits20' # all issues

vols = range(14, 25) # vol14: 2010, vol24: 2020
year = 2010
issues = range(1,7)

urls = []

for vol in vols:
    if vol <= 19:
        for issue in range(1, 5):
            url = base_url + "/" + str(vol) + "/" + str(issue) + '?nav=tocList'
            urls.append((year, vol, issue, url))
    else:
        for issue in range(1, 7):
            url = base_url + "/" + str(vol) + "/" + str(issue) + '?nav=tocList'
            urls.append((year, vol, issue, url))
    year += 1

book_urls = pd.DataFrame(urls, columns = ["Year", "Vol", "Issue", "BookUrl"])
book_urls["BookId"] = book_urls["Year"].astype(str) + "-" + book_urls["Vol"].astype(str) + "-" + book_urls["Issue"].astype(str)
book_urls["Journal"] = "JITS"

In [5]:
article_base_url = 'https://www.tandfonline.com'
titles = []
a_urls = []
authors = []
bookids = []

for row in book_urls.iterrows():
    bookid = row[1]['BookId']
    bookurl = row[1]['BookUrl']
    page = requests.get(bookurl, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    for article in soup.find_all('div', {"class": "articleEntry"}):
        for div in article.find_all('div'):           
            if "art_title" in div['class']:
                for a in div.find_all('a'):
                    a_urls.append(article_base_url + a['href'])
                    for span in a.find_all('span'):
                        titles.append(span.get_text())
                        bookids.append(bookid)
            if "tocAuthors" in div['class']:
                for divl in div.find_all('div', {"class": "articleEntryAuthor"}):
                    author = []
                    for a in divl.find_all('a'): 
                        author.append(a.get_text())
                    authors.append(author)
    time.sleep(2)

In [11]:
paper_urls = pd.DataFrame(list(zip(bookids, titles, a_urls, authors)), columns = ['BookId', 'Titles', 'Urls', 'Authors'])
paper_urls['Authors'] = paper_urls['Authors'].apply(lambda x: ','.join(map(str, x)))

In [23]:
# to be changed
conn = sql.connect("jits.db")
book_urls.to_sql("book_urls", conn, index = False)
paper_urls.to_sql("paper_urls", conn, index = False)
conn.close()